# 1. Imports e Definindo os Classificadores

In [297]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.covariance import empirical_covariance
from numpy.linalg import pinv

## 1.1 Classificador NN com distância euclidiana:

In [3]:
class MateusLindoNN():

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        

    def predict(self, X_test):
        labels = []
        for vec in X_test:
            dist = [(np.linalg.norm(vec - self.X_train[ii]), l)
                    for ii, l in enumerate(self.y_train)]  # d:distance, l: label
            nearestL = min(dist, key=lambda x: x[0])[1]
            labels.append(nearestL)
        return labels

    def score(self, X_test, y_test):
        return accuracy_score(y_test, self.predict(X_test))

## 1.2 Classificador NN com distância de Mahanalobis modificada:

Na literatura e em implementações como a Scikit-learn, o KNN com distância de Mahanalobis é feito utilizando a matriz de covariância de todo o conjunto de treinamento, e isso faz sentido para o caso do KNN com K>1, mas para o caso do K=1 é possível implementar com uma matriz de covariância distinta para cada classe. Na comparação dos classificadores implementados com os da biblioteca Scikit-Learn.

In [331]:
class MateusLindoNNMaha():
    def fit(self, X_train, y_train):
        self.classes = np.unique(y_train)
        self.X_train = X_train
        self.y_train = y_train
        dataClasses = {}
        self.means = {}
        self.covs = {}
        self.ranks = {}
        self.conds = {}
        self.invs = {}
        self.cov = np.cov(X_train, rowvar= False)
        self.inv = pinv(self.cov)
        for clas in self.classes:
            dataClasses[clas] = X_train[y_train == clas]
            self.means[clas] = np.mean(dataClasses[clas],axis=0)  # mean of every feature for class 'clas' 
            self.covs[clas] = np.cov(dataClasses[clas],rowvar=False)  #covariance matrix of class
            self.ranks[clas] = np.linalg.matrix_rank(dataClasses[clas]) # rank of the data matrix for each class
            self.conds[clas] = np.linalg.cond(dataClasses[clas])  # condition number of matrix
            self.invs[clas] = pinv(self.covs[clas]) #inverse of covariance

    def predict(self, X_test):
        labels = []
        for vec in X_test:
            dist = []
            #dist = [( (vec - self.X_train[ii])@self.invs[l]@(vec - self.X_train[ii]).T , l) for ii, l in enumerate(self.y_train)]  # d:distance, l: label 
            dist = [( (vec - self.X_train[ii])@self.invs[l]@(vec - self.X_train[ii]).T , l) for ii, l in enumerate(self.y_train)]  # d:distance, l: label 
            nearestL = min(dist, key=lambda x: x[0])[1]
            labels.append(nearestL)
        return labels

    def score(self, X_test, y_test):
        return accuracy_score(y_test, self.predict(X_test))

## 1.3 Distância Mínima ao Centróide:

In [234]:
class MateusLindoNC():

    def fit(self, X_train, y_train):
        centers = []
        classes = {c: [] for c in np.unique(y_train)}
        for (x, y) in zip(X_train, y_train):
            classes[y].append(x)
        for k in classes:
            center = np.mean(classes[k], axis=0)
            centers.append((center,k))
        self.centers = centers
    
    def predict(self, X_test):
        labels = []
        for vec in X_test:
            distLabel = [(np.linalg.norm(vec - center[0]), center[1]) for center in self.centers]  # tuple: (distance, label)
            nearest = min(distLabel, key=lambda x: x[0])[1]
            labels.append(nearest)
        return labels
    
    def score(self, X_test, y_test):
        return accuracy_score(y_test, self.predict(X_test))

## 1.4 Classificador Quadrático Gaussiano:

### Sem regularização:

In [318]:
class MateusLindoCQG():
    
    def fit(self, X_train, y_train):
        self.classes = np.unique(y_train)
        dataClasses = {}
        self.means = {}
        self.covs = {}
        self.ranks = {}
        self.conds = {}
        self.invs = {}
        for clas in self.classes:
            dataClasses[clas] = X_train[y_train == clas]
            self.means[clas] = np.mean(dataClasses[clas],axis=0)  # mean of every feature for class 'clas' 
            self.covs[clas] = np.cov(dataClasses[clas], rowvar=False)  #covariance matrix of class
            self.ranks[clas] = np.linalg.matrix_rank(dataClasses[clas]) # rank of the data matrix for each class
            self.conds[clas] = np.linalg.cond(dataClasses[clas])  # condition number of matrix
            self.invs[clas] = np.linalg.inv(self.covs[clas]) #inverse of covariance
        
            
    def predict(self, X_test):
        labels = []
        for vec in X_test:
            #  tuple (pred_value, class) 
            preds = [ ((vec - self.means[c])@self.invs[c]@(vec - self.means[c]).T + np.log(np.linalg.det(self.covs[c])), c) for c in self.classes]
            label = min(preds, key = lambda x: x[0])[1]
            labels.append(label)
        return labels
    
    def score(self, X_test, y_test):
        return accuracy_score(y_test, self.predict(X_test))

### Com Regularização de Friedman

In [350]:
class MateusLindoCQGF():
    
    def __init__(self, alpha = 0.5):
        self.alpha = alpha
        
    def fit(self, X_train, y_train):
        self.classes = np.unique(y_train)
        dataClasses = {}
        self.means = {}
        self.covs = {}
        self.ranks = {}
        self.conds = {}
        self.invs = {}
        need_reg = False

        for clas in self.classes:
            dataClasses[clas] = X_train[y_train == clas]
            self.means[clas] = np.mean(dataClasses[clas],axis=0)  # mean of every feature for class 'clas' 
            self.covs[clas] = np.cov(dataClasses[clas], rowvar=False)  #covariance matrix of class
            self.ranks[clas] = np.linalg.matrix_rank(self.covs[clas]) # rank of the data matrix for each class
            self.conds[clas] = np.linalg.cond(self.covs[clas])  # condition number of matrix
            if self.ranks[clas] == len(self.covs[clas]): # Caso seja de rank completo
                self.invs[clas] = np.linalg.inv(self.covs[clas]) #inverse of covariance
            else: need_reg = True
        
        if need_reg:
            covX = np.cov(X_train,rowvar=False)
            for clas in self.classes:
                self.covs[clas] = (1-self.alpha) * self.covs[clas] + (self.alpha)*covX
                self.invs[clas] = np.linalg.inv(self.covs[clas])    
            
    def predict(self, X_test):
        labels = []
        for vec in X_test:
            #  tuple (pred_value, class) 
            # reg
            preds = [ ((vec - self.means[c])@self.invs[c]@(vec - self.means[c]).T + np.log(np.linalg.det(self.covs[c])), c) for c in self.classes] #com regularização
            label = min(preds, key = lambda x: x[0])[1]
            labels.append(label)
        return labels
    
    def score(self, X_test, y_test):
        return accuracy_score(y_test, self.predict(X_test))

## 1.5 Importando os dados: 

In [236]:
df = pd.read_csv('parkinsons.data',header=0,index_col=0)
dfY = df['status']
dfX = df.drop('status',axis=1)

# 2. Avaliando sem escalar os dados

Neste passo avaliamos o desempenho dos classificadores implementados e comparamos o desempenho deles com os mesmos classificadores, mas com a implementação da biblioteca Scikit-Learn. Os objetivos deste passo são:
1.  Avaliar se a nossa implementação está correta ao comparar com implementações mais sólidas.
2.  Obter os mesmos resultados nos classificadores Vizinho mais Próximo, Classificador Quadrático Gaussiano e Centróide mais Próximo.
3.  Comparar o desempenho da nossa implementação diferenciada do Vizinho mais Próximo com distância de Mahalanobis com a implementação padrão do Scikit-Learn.

Para verificar os resultados acima serão feitas 1000 rodadas do experimento, cada rodada vai nos dar um valor de acurácia para cada classificador, esta acurácia será armazenada em um vetor para cada classificador que será utilizado mais a frente.
Podemos instanciar os classificadores antes, com exceção do NN com distância de Mahalanobis do SciKit-Learn, pois ele necessita de um parametro *V* que é a matriz de covariância.

## 2.1 Instanciando os Classificadores:

In [340]:
NNclass = KNeighborsClassifier(n_neighbors=1)
Ncenter = NearestCentroid()
QClass = QuadraticDiscriminantAnalysis()
QClassF = QuadraticDiscriminantAnalysis(reg_param=0.5)
mateusNN = MateusLindoNN()
mateusNC = MateusLindoNC()
mateusCQG =  MateusLindoCQG()
mateusCQGF = MateusLindoCQGF()
mateusMaha = MateusLindoNNMaha()

In [341]:
X_train, X_test, y_train, y_test = train_test_split(dfX,dfY,test_size = 0.2)

QClass.fit(X_train,y_train)
Ncenter.fit(X_train,y_train)
NNclass.fit(X_train,y_train)
NNMaha.fit(X_train, y_train)
QClassF.fit(X_train, y_train)
mateusNN.fit(X_train.values, y_train.values)
mateusNC.fit(X_train.values, y_train.values)
mateusCQG.fit(X_train.values, y_train.values)
mateusCQGF.fit(X_train.values, y_train.values)
mateusMaha.fit(X_train.values, y_train.values)

print(NNclass.score(X_test,y_test),Ncenter.score(X_test,y_test), QClass.score(X_test,y_test),QClassF.score(X_test,y_test), NNMaha.score(X_test,y_test))
print(mateusNN.score(X_test.values,y_test.values), mateusNC.score(X_test.values,y_test.values), mateusCQG.score(X_test.values,y_test.values),mateusCQGF.score(X_test.values,y_test.values), mateusMaha.score(X_test.values,y_test.values) )

0.8205128205128205 0.6923076923076923 0.8717948717948718 0.7435897435897436 0.8974358974358975
0.8205128205128205 0.6923076923076923 0.8717948717948718 0.7948717948717948 0.8461538461538461


/home/mateuspontes/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


## 2.2 Simulação de Monte Carlo:

In [343]:
skNN =[]
skNC = []
skCQG = []
skCQGF = []
skNNMaha = []
acMateusNN = []
acMateusNC = []
acMateusCQG = []
acMateusCQGF = []
acMateusNNMaha = []

for ii in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(dfX,dfY,test_size = 0.2)
    NNMaha = KNeighborsClassifier(n_neighbors=1, metric='mahalanobis', metric_params={'V': np.cov(X_train,rowvar=False)})
    ## SKLEARN ##########
    QClass.fit(X_train,y_train)
    QClassF.fit(X_train,y_train)
    Ncenter.fit(X_train,y_train)
    NNclass.fit(X_train,y_train)
    NNMaha.fit(X_train, y_train)
    ## HANDCODED #########
    mateusNN.fit(X_train.values, y_train.values)
    mateusNC.fit(X_train.values, y_train.values)
    mateusCQG.fit(X_train.values, y_train.values)
    mateusCQGF.fit(X_train.values, y_train.values)
    mateusMaha.fit(X_train.values, y_train.values)
    ### Acuracy Vectors ######
    skNN.append(NNclass.score(X_test,y_test))
    skNC.append(Ncenter.score(X_test,y_test))
    skCQG.append(QClass.score(X_test,y_test))
    skCQGF.append(QClassF.score(X_test,y_test))
    skNNMaha.append(NNMaha.score(X_test,y_test))
    acMateusNN.append(mateusNN.score(X_test.values,y_test.values))
    acMateusNC.append(mateusNC.score(X_test.values,y_test.values)) 
    acMateusCQG.append(mateusCQG.score(X_test.values,y_test.values))
    acMateusCQGF.append(mateusCQGF.score(X_test.values,y_test.values))
    acMateusNNMaha.append(mateusMaha.score(X_test.values,y_test.values))

/home/mateuspontes/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


## 2.3 Avaliação dos classificadores:

### NN e DMC
Os classificadores NN e DMC implementados possuem os mesmos valores de acurácia dos algoritmos da biblioteca Scikit-Learn, como podemos ver abaixo:

In [344]:
print(skNN == acMateusNN)
print(skNC == acMateusNC)

True
True


### CQG sem Regularização
* Já o nosso CQG possui desempenho similar na média, 0.8790512820512822 do Scikit-Learn contra 0.8787435897435899 da nossa implementação. 
* Além disso os valores variam no máximo de 8% entre as implementações. 
   
É possível concluir que a diferença entre as implementações está relacionada ao cálculo da matriz de covariância, apesar de saber do problema do nosso algoritmo não foi possível alcançar um desempenho idêntico mesmo utilizando a Regularização de Friedman e a matriz de covariância Pooled. Para efeito de demonstração faremos outra simulação de Monte Carlo mais a frente comparado estes algoritmos.

In [347]:
print(np.mean(skCQG),np.mean(acMateusCQG))
print(np.allclose(skCQG,acMateusCQG,atol=0.08))

0.8786666666666668 0.8766923076923079
True


## CQG com Regularização

O CQG com regularização de Friedman possui desempenho inferior ao CQG sem regularização, com a diferença entre as médias chegando a 1,5%. Apesar desse adendo é importante frisar que a regularização efetuada pelo Scikit-Learn é diferente da de Friedman, ja que ela utiliza a equação abaixo:

$$ (1- reg\_param)\Sigma + (reg\_param) \, I_{n\_features} $$
Onde $\Sigma$ é a matriz de covariância. Já a nossa implementação utiliza a regularização dada pela equação abaixo:
$$ \Sigma_k = (1- \alpha) \Sigma_k + \alpha \Sigma $$
Onde $\Sigma_k$ é a matriz de covariância da classe $K$

In [349]:
print(np.mean(skCQGF),np.mean(acMateusCQGF))
print(np)

0.7651282051282052 0.7500256410256411


## NN com Distância de Mahalanobis

Já para a comparação do nosso algoritmo do NN com distância de Mahalanobis foi possível verificar que existe a nossa implementação é 1% superior na média em relação ao algoritmo de NN com distância de Mahalanobis padrão do Scikit-Learn. Um ponto a ser considerado é a diferença no cálculo da inversa da matriz de covariância da biblioteca do Scikit-Learn, como mostrado na comparação dos CQGs, um ponto a ser estudado posteriormente é se utilizando o cálculo semelhante ao do Scikit-Learn é possível melhorar ainda mais o desempenho desta versão diferenciada do algoritmo.

* Comparando as médias obtemos os valores de 0.8702820512820515 (Scikit-Learn) e 0.8801025641025643 (Implementação diferenciada).

In [348]:
print(np.mean(skNNMaha),np.mean(acMateusNNMaha))

0.8695897435897437 0.857769230769231


## Comparação dos algoritmos:

Para fazermos uma comparação mais minuciosa dos classificadores podemos construir uma tabela com os valores da média, mediana, mínimo e máximo, desvio padrão, sensibilidade e especifidade. 

# 2. Procurando o Melhor Escalador para cada Classificador

In [109]:
#Classificadores
NNclass = KNeighborsClassifier(n_neighbors=1)
Ncenter = NearestCentroid()
QClass = LinearDiscriminantAnalysis()
#Pipelines:
stepsNN = [('scaler', StandardScaler()),
         ('classifier', NNclass)]
stepsNC = [('scaler', StandardScaler()),
         ('classifier', Ncenter)]
stepsQC = [('scaler', StandardScaler()),
         ('classifier', QClass)]
pipelineNN = Pipeline(stepsNN)
pipelineNC = Pipeline(stepsNC)
pipelineQC = Pipeline(stepsQC)
#Fits e Scores
pipelineNN.fit(X_train,y_train)
pipelineNC.fit(X_train,y_train)
pipelineQC.fit(X_train,y_train)
print(pipelineNN.score(X_test,y_test), pipelineNC.score(X_test,y_test), pipelineQC.score(X_test,y_test))


0.9743589743589743 0.7435897435897436 0.8717948717948718


In [114]:
#Classificadores
NNclass = KNeighborsClassifier(n_neighbors=1)
Ncenter = NearestCentroid()
QClass = LinearDiscriminantAnalysis()
#Pipelines:
stepsNN = [('scaler', StandardScaler()),
         ('classifier', NNclass)]
stepsNC = [('scaler', MinMaxScaler()),
         ('classifier', Ncenter)]
stepsQC = [('scaler', Normalizer()),
         ('classifier', QClass)]
pipelineNN = Pipeline(stepsNN)
pipelineNC = Pipeline(stepsNC)
pipelineQC = Pipeline(stepsQC)
#Fits e Scores
pipelineNN.fit(X_train,y_train)
pipelineNC.fit(X_train,y_train)
pipelineQC.fit(X_train,y_train)
print(pipelineNN.score(X_test,y_test), pipelineNC.score(X_test,y_test), pipelineQC.score(X_test,y_test))


0.9743589743589743 0.7948717948717948 0.8974358974358975
